In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import folium
import os

import warnings
warnings.filterwarnings('ignore')

import plotly.graph_objects as go

In [ ]:
data = pd.read_csv("../input/covid19-in-india/covid_19_india.csv")
data.head()

In [ ]:
data['TotalCases'] = data['ConfirmedIndianNational']+data['ConfirmedForeignNational']
data['ActiveCases'] = data['TotalCases']-data['Cured']-data['Deaths']

In [ ]:
date_wise = data.groupby(['Date','State/UnionTerritory','TotalCases'])['Cured','Deaths','ActiveCases'].sum().reset_index().sort_values('TotalCases',ascending=False)

In [ ]:

fig = px.bar(date_wise,height=700,x='Date',y='TotalCases',hover_data =['State/UnionTerritory','ActiveCases','Deaths','Cured'],color='TotalCases')
fig.show()

In [ ]:
data_today= data[data['Date']=='27/03/20']
data_new = data_today.groupby(['State/UnionTerritory','Cured','Deaths'])['TotalCases'].sum().reset_index().sort_values('TotalCases',ascending=False)
data_new

In [ ]:
data_today.columns

In [ ]:
fig = px.bar(data_today,x='State/UnionTerritory',y='TotalCases',height=600,color = 'ActiveCases')
fig.show()

In [ ]:
data_population = pd.read_csv("../input/covid19-in-india/population_india_census2011.csv")
data_population.head()

In [ ]:
data_population.columns

In [ ]:
#merging of data_today and data_population
data_with_population = data_population.merge(data_new,left_on='State / Union Territory',right_on='State/UnionTerritory')

In [ ]:
data_with_population = data_with_population.drop(labels=['State/UnionTerritory','Sno'],axis=1)

In [ ]:
data_with_population.head()

In [ ]:
data_with_population = data_with_population.sort_values('Population',ascending = False)
data_with_population             # by this we can understand currently there is no relation between population and number of cases

In [ ]:
data_hospitalbeds = pd.read_csv("../input/covid19-in-india/HospitalBedsIndia.csv")
data_hospitalbeds.head()

In [ ]:
data_population_hospitalbeds = data_with_population.merge(data_hospitalbeds,left_on='State / Union Territory',right_on='State/UT')

In [ ]:
data_population_hospitalbeds = data_population_hospitalbeds.drop(labels=['State/UT','Sno'],axis =1)

In [ ]:
data_population_hospitalbeds = data_population_hospitalbeds.sort_values('TotalCases',ascending = False)

In [ ]:
data_population_hospitalbeds.dtypes

In [ ]:
data_population_hospitalbeds= data_population_hospitalbeds.drop(labels=['Unnamed: 12','Unnamed: 13'],axis=1)
data_population_hospitalbeds.head()

In [ ]:
data_population_hospitalbeds.fillna(value=0)

In [ ]:
data_population_hospitalbeds['NumUrbanHospitals_NHP18'] = data_population_hospitalbeds['NumUrbanHospitals_NHP18'].astype('int')
data_population_hospitalbeds['NumRuralHospitals_NHP18'] = data_population_hospitalbeds['NumRuralHospitals_NHP18'].astype('int')
#data_population_hospitalbeds['NumSubDistrictHospitals_HMIS'] = data_population_hospitalbeds['NumSubDistrictHospitals_HMIS'].astype('int')
data_population_hospitalbeds['NumDistrictHospitals_HMIS'] = data_population_hospitalbeds['NumDistrictHospitals_HMIS'].astype('int')


In [ ]:
data_population_hospitalbeds['TotalHospitals_inState']= data_population_hospitalbeds['NumUrbanHospitals_NHP18']+data_population_hospitalbeds['NumRuralHospitals_NHP18']+data_population_hospitalbeds['NumDistrictHospitals_HMIS']

data_population_hospitalbeds.head()

In [ ]:
fig = px.bar(data_population_hospitalbeds,x='State / Union Territory',y='TotalCases',height=600,color = 'TotalCases')
fig.show()

In [ ]:
fig = px.bar(data_population_hospitalbeds,x='State / Union Territory',y='TotalHospitals_inState', height=600, color = 'TotalCases')
fig.show()

In [ ]:
#by the above two graphs Kerala and Mumbai are having very less hosiptals and more number of positive Cases

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
data = data.drop(labels='Sno',axis=1)

In [ ]:
data = data.drop(labels='Time',axis=1)

In [ ]:
data.Date = pd.to_datetime(data.Date)
data.Date = data.Date.astype(int)
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['State/UnionTerritory'] = le.fit_transform(data['State/UnionTerritory'])

In [ ]:
Y = data['Deaths']
X = data.drop(['Deaths'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100,random_state=10)
model.fit(X_train,y_train)

In [ ]:
prediction = model.predict(X_test)
prediction

In [ ]:
print(y_test)

In [ ]:
from sklearn.metrics import  accuracy_score
print(accuracy_score(y_test, prediction))

In [ ]:
#lets predict the deaths on 28th March...

d = {'Date': ['28/03/20'], 'State/UnionTerritory': 'Karnataka','ConfirmedIndianNational':55,'ConfirmedForeignNational':0,'Cured':3,'TotalCases':55,'ActiveCases':52 }
df = pd.DataFrame(data=d)
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['State/UnionTerritory'] = le.fit_transform(df['State/UnionTerritory'])

In [ ]:
df.Date= pd.to_datetime(df.Date)
df.Date = df.Date.astype(int)
df.head()

In [ ]:
pred = model.predict(df)
pred              # Death =2